# SV signature analysis tutorial

## 0. Overview

```python
import viola
pcawg_bedpe=viola.read_bedpe_multi('./resources/pcawg/')
bed_fragile = viola.read_bed('./resources/annotation/fragile_site.hg19.bed')
bedgraph_timing = viola.read_bed('./resources/annotation/replication_timing.bedgraph')
pcawg_bedpe.annotate_bed(bed=bed_fragile, annotation='fragile', how='flag')
pcawg_bedpe.annotate_bed(bed=bedgraph_timing, annotation='timing', how='value')
pcawg_bedpe.calculate_info('(${timingleft} + ${timingright}) / 2', 'timing')
feature_matrix = pcawg_bedpe.classify_manual_svtype(definitions='./resources/definitions/sv_class_definition.txt', return_data_frame=True)
```

## 1. Import

In [1]:
import viola

## 2. Reading BEDPE files into viola.MultiBedpe object

In [2]:
pcawg_bedpe=viola.read_bedpe_multi('./resources/pcawg/', exclude_empty_cases=True)

This code reads all BEDPE files in the `./resources/pcawg` directory into MultiBedpe class (See [MultiBedpe](https://dermasugita.github.io/ViolaDocs/docs/html/reference/api/viola.MultiBedpe.html)). Because the BEDPE files created by PCAWG have the `svclass` columns, we passed it to the `svtype_col_name` argument. Some BEDPE files did not have any SV records. This time we will exclude these by setting ``exclued_empty_cases=True``.

## 3. Reading BED/BEDGRAPH files for annotation

In [3]:
bed_fragile = viola.read_bed('./resources/annotation/fragile_site.hg19.bed')
bed_timing = viola.read_bed('./resources/annotation/replication_timing.bedgraph')

Reading BED and BEDGRAPH files required for custom SV classification. At the moment we do not make a clear distinction between BED files and BEDGRAPH files. This is because only the first four columns of these files are used for annotation purposes in the first place.

`fragile_site.hg19.bed` is a BED file specifying the known common fragile site (CFS) regions. <br>
`replication_timing.bedgraph` is a BEDGRAPH file which records the replication timing for each genome coordinate divided into bins.

These files were built according to the [PCAWG paper](https://www.nature.com/articles/s41586-019-1913-9#Sec20).

## 4. Annotating SV

In [4]:
pcawg_bedpe.annotate_bed(bed=bed_fragile, annotation='fragile', how='flag')
pcawg_bedpe.annotate_bed(bed=bed_timing, annotation='timing', how='value')

In this step, we annotate `pcawg_bedpe` with the Bed object we’ve just loaded. After annotation, new INFO – "fragileleft", "fragileright", "timingleft", and "timingright" – will be added. Because two breakends form a single SV, "left" and "right" suffix are added. When `how='flag'`, annotate True/False according wether each breakend is in the range in the Bed (4th column of the Bed is ignored). When `how='value'`, annotate the value of 4th column of Bed if the breakends hit.

## 5. Get Average values of Replication Timing

In [5]:
pcawg_bedpe.calculate_info('(${timingleft} + ${timingright}) / 2', 'timing')

To get representative values of replication timing for each SV breakpoints, we decided to take mean values of two breakends. This code adds new INFO named "timing" by calculating mean values of "timingleft" and "timingright".

## 6. Classify SV and Generate Feature Matrix

In [6]:
feature_matrix = pcawg_bedpe.classify_manual_svtype(definitions='./resources/definitions/sv_class_definition.txt', return_data_frame=True)

Finally, we classified SV according to its type, size, fragile site, and replication timing. Classification criteria are written in `sv_class_definition.txt`. Syntax of this file is explained in the [Viola Documentation](https://dermasugita.github.io/ViolaDocs/docs/html/signature_analysis.html).

If `return_data_frame=True`, counts of each custom SV class for each patients are returned as pandas.DataFrame.

Now we successfully obtained feature matrix with custom SV classification!